In [1]:
import numpy as np
import pandas as pd

In [6]:
df = pd.read_csv("./task_2022/train_data_swc.csv")

Light exploration

In [7]:
df.head()

,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10,...,x_100,x_101,x_102,x_103,x_104,x_105,x_106,x_107,x_108,y
0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,3
1,0.2,0.8,0.0,0.0,0.0,0.0,0.6,0.2,0.0,0.0,...,0.2,0.0,0.8,0.0,0.0,0.0,0.0,0.0,0.0,9
2,0.0,0.0,0.0,0.4,0.0,0.0,3.2,0.0,0.0,0.0,...,0.0,0.0,0.6,0.0,0.0,0.0,0.0,0.0,0.0,3
3,0.0,0.0,0.2,0.0,0.0,0.0,0.2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
4,0.0,0.0,0.4,0.0,0.0,0.0,0.4,0.0,0.0,0.6,...,0.0,0.0,0.0,0.6,0.2,0.0,0.0,0.2,0.0,9


In [8]:
target = "y"

In [13]:
# Display basic information about the DataFrame
print("Data Types:")
print(df.dtypes == np.float64)
print("\nData Shape:")
print(df.shape)
print("\nNull Value Summary:")
print(df.isnull().sum().sort_values())

Data Types:
x_1       True
x_2       True
x_3       True
x_4       True
x_5       True
         ...  
x_105     True
x_106     True
x_107     True
x_108     True
y        False
Length: 109, dtype: bool

Data Shape:
(62500, 109)

Null Value Summary:
x_1     0
x_79    0
x_78    0
x_77    0
x_76    0
       ..
x_31    0
x_30    0
x_29    0
x_40    0
y       0
Length: 109, dtype: int64


Clean data with no nulls, and X is float64.

In [15]:
# Summary statistics for numeric columns
numeric_columns = df.select_dtypes(include='number').columns
numeric_summary = df[numeric_columns].describe()
print("\nSummary Statistics for Numeric Columns:")
print(numeric_summary)


Summary Statistics for Numeric Columns:
                x_1           x_2           x_3           x_4           x_5  \
count  62500.000000  62500.000000  62500.000000  62500.000000  62500.000000   
mean       0.044384      0.114995      0.042627      0.171478      0.047898   
std        0.156245      0.266926      0.209134      0.481939      0.203279   
min        0.000000      0.000000      0.000000      0.000000      0.000000   
25%        0.000000      0.000000      0.000000      0.000000      0.000000   
50%        0.000000      0.000000      0.000000      0.000000      0.000000   
75%        0.000000      0.200000      0.000000      0.200000      0.000000   
max        7.600000      6.400000      9.600000      6.000000      7.600000   

                x_6           x_7           x_8           x_9          x_10  \
count  62500.000000  62500.000000  62500.000000  62500.000000  62500.000000   
mean       0.157261      0.510973      0.030339      0.052819      0.096179   
std       

In [18]:
# Correlation with the target variable
correlation = df.corr()[target]
correlation = correlation.drop(target)
print("\nCorrelation with Target Variable:")
print(correlation.sort_values())


Correlation with Target Variable:
x_11   -0.365018
x_72   -0.330005
x_47   -0.293908
x_24   -0.265928
x_81   -0.258775
          ...   
x_74    0.230232
x_53    0.235274
x_76    0.241530
x_50    0.253853
x_87    0.261362
Name: y, Length: 108, dtype: float64


In [20]:
unique_classes = df[target].nunique()
print(f"Number of unique classes in {target}: {unique_classes}")

Number of unique classes in y: 9


In [21]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import VarianceThreshold

# Split data into features(X) and target(y)
X = df.drop(columns=[target])
y = df[target]

# Encoding the target variable
le = LabelEncoder()
y = le.fit_transform(y)

In [23]:
# Feature selection to remove redundant features
selector = VarianceThreshold()
X = selector.fit_transform(X)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a lgb.Dataset
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# Set parameters
params = {
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'num_class': 9,  # number of classes
    'metric': 'multi_logloss',
    'learning_rate': 0.1,
    'num_leaves': 31,
    'verbose': -1,
}

# Train the model
model = lgb.train(params, train_data, valid_sets=test_data, num_boost_round=5000, early_stopping_rounds=100)

# Make prediction
y_pred = model.predict(X_test)

/home/serhat/Documents/cases/science_wave_task/.venv/lib/python3.10/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	valid_0's multi_logloss: 1.5962
Training until validation scores don't improve for 100 rounds
[2]	valid_0's multi_logloss: 1.42292
[3]	valid_0's multi_logloss: 1.29602
[4]	valid_0's multi_logloss: 1.19617
[5]	valid_0's multi_logloss: 1.1157
[6]	valid_0's multi_logloss: 1.04895
[7]	valid_0's multi_logloss: 0.992085
[8]	valid_0's multi_logloss: 0.943642
[9]	valid_0's multi_logloss: 0.90216
[10]	valid_0's multi_logloss: 0.866929
[11]	valid_0's multi_logloss: 0.834865
[12]	valid_0's multi_logloss: 0.807091
[13]	valid_0's multi_logloss: 0.782618
[14]	valid_0's multi_logloss: 0.760877
[15]	valid_0's multi_logloss: 0.741382
[16]	valid_0's multi_logloss: 0.723707
[17]	valid_0's multi_logloss: 0.708474
[18]	valid_0's multi_logloss: 0.69414
[19]	valid_0's multi_logloss: 0.682162
[20]	valid_0's multi_logloss: 0.670756
[21]	valid_0's multi_logloss: 0.660028
[22]	valid_0's multi_logloss: 0.65025
[23]	valid_0's multi_logloss: 0.641548
[24]	valid_0's multi_logloss: 0.633421
[25]	valid_0's multi_l

In [27]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Convert into binary outputs
y_pred_binary = np.argmax(y_pred, axis=1)

# Print classification report
print(classification_report(y_test, y_pred_binary, digits=4))

# Print confusion matrix
print(confusion_matrix(y_test, y_pred_binary))

# Print accuracy
print("Accuracy: ", accuracy_score(y_test, y_pred_binary))


              precision    recall  f1-score   support

           0     0.6984    0.6011    0.6461       366
           1     0.7341    0.8511    0.7883      3191
           2     0.6255    0.5387    0.5789      1628
           3     0.7904    0.5360    0.6388       584
           4     0.9837    0.9611    0.9722       565
           5     0.9479    0.9470    0.9475      2904
           6     0.7538    0.6701    0.7095       585
           7     0.9060    0.9174    0.9117      1682
           8     0.8665    0.8804    0.8734       995

    accuracy                         0.8184     12500
   macro avg     0.8118    0.7670    0.7851     12500
weighted avg     0.8171    0.8184    0.8148     12500

[[ 220   14    3    0    0   22   11   34   62]
 [   1 2716  378   42    4    7   31    8    4]
 [   1  681  877   29    0    3   28    5    4]
 [   0  174   76  313    1   13    4    3    0]
 [   0   18    0    0  543    0    2    1    1]
 [  10   16    2    8    0 2750   33   44   41]
 [   8 

In [26]:
from sklearn.metrics import log_loss

# Calculate log loss
loss = log_loss(y_test, y_pred)

print("Log loss: ", loss)

Log loss:  0.49063796166014445


Simple approach with stacking using lgbm the nn, rf, svm models in addition of feature selection using rfe

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.svm import SVC
from sklearn.feature_selection import RFE
from sklearn.pipeline import make_pipeline
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, log_loss, classification_report, confusion_matrix

# Split data into features(X) and target(y)
X = df.drop(columns=[target])
y = df[target]

# Feature scaling for better performance of SVM
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# Using RFE for feature selection
selector = RFE(estimator=RandomForestClassifier(), n_features_to_select=50, step=10)
X_selected = selector.fit_transform(X, y)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# Define the base models
level0 = list()
level0.append(('nn', MLPClassifier(max_iter=1000)))
level0.append(('rf', RandomForestClassifier()))
level0.append(('svm', SVC(probability=True)))  # `probability=True` to enable use of predict_proba for SVM

# Define meta learner model
level1 = LGBMClassifier(objective='multiclass', num_class=9)

# Define the stacking ensemble
model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)

# Fit the model on all available data
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)  # For log loss

In [ ]:
# Print classification metrics
print(classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Accuracy: ", accuracy_score(y_test, y_pred))
print("Log loss: ", log_loss(y_test, y_pred_proba))

Stacking using decision tree where base models are nn, rf, svm, lgbm, extra, ridge

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score

# Define base models
nn = MLPClassifier(max_iter=1000)
rf = RandomForestClassifier()
svm = SVC(probability=True)
lgbm = LGBMClassifier(objective='multiclass', num_class=9)
extra_trees = ExtraTreesClassifier()
ridge = RidgeClassifier()

# Grid search parameters
# Here, for simplicity, we will only tune one parameter for each model.
# In real scenario, consider tuning more parameters and using a more advanced method
nn_params = {'alpha': [0.0001, 0.001, 0.01]}  # MLP parameters
rf_params = {'n_estimators': [50, 100, 200]}  # RandomForest parameters
svm_params = {'C': [0.1, 1, 10]}  # SVM parameters
lgbm_params = {'num_leaves': [31, 50, 100]}  # LGBM parameters
extra_trees_params = {'n_estimators': [50, 100, 200]}  # ExtraTrees parameters
ridge_params = {'alpha': [0.1, 1, 10]}  # Ridge parameters

# Define GridSearchCV for each base model
gs_nn = GridSearchCV(nn, nn_params, cv=5)
gs_rf = GridSearchCV(rf, rf_params, cv=5)
gs_svm = GridSearchCV(svm, svm_params, cv=5)
gs_lgbm = GridSearchCV(lgbm, lgbm_params, cv=5)
gs_extra_trees = GridSearchCV(extra_trees, extra_trees_params, cv=5)
gs_ridge = GridSearchCV(ridge, ridge_params, cv=5)

# Define base models list
level0 = list()
level0.append(('nn', gs_nn))
level0.append(('rf', gs_rf))
level0.append(('svm', gs_svm))
level0.append(('lgbm', gs_lgbm))
level0.append(('extra_trees', gs_extra_trees))
level0.append(('ridge', gs_ridge))

# Define meta learner model
level1 = DecisionTreeClassifier()

# Define the stacking ensemble
model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5)

# Fit the model
model.fit(X_train, y_train)

# Compute cross-validation score
cv_scores = cross_val_score(model, X_selected, y, cv=5)

print("Cross-validation scores: ", cv_scores)
print("Mean cross-validation score: ", np.mean(cv_scores))

# Make predictions
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)  # For log loss

In [ ]:
# Print classification metrics
print(classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Accuracy: ", accuracy_score(y_test, y_pred))
print("Log loss: ", log_loss(y_test, y_pred_proba))